In [1]:
import soundfile as sf
from IPython.display import Audio
from IPython.core.display import HTML
import torch
from nemo.collections.tts.models.base import SpectrogramGenerator, Vocoder
from voicefixer import VoiceFixer

import tempfile

In [2]:
# Download and load the pretrained tacotron2 model
spec_generator = SpectrogramGenerator.from_pretrained("tts_en_fastpitch_multispeaker")

# Download and load the pretrained waveglow model
vocoder = Vocoder.from_pretrained("tts_en_hifitts_hifigan_ft_fastpitch")

voicefixer = VoiceFixer()

[NeMo I 2024-04-28 18:24:59 nemo_logging:381] Found existing object /home/you/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_multispeaker/b96451578e2b9a39d62cfc642bc5ffcf/tts_en_fastpitch_multispeaker.nemo.
[NeMo I 2024-04-28 18:24:59 nemo_logging:381] Re-using file from: /home/you/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_multispeaker/b96451578e2b9a39d62cfc642bc5ffcf/tts_en_fastpitch_multispeaker.nemo
[NeMo I 2024-04-28 18:24:59 nemo_logging:381] Instantiating model from pre-trained checkpoint


 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
[NeMo W 2024-04-28 18:25:34 nemo_logging:393] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2024-04-28 18:25:34 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/HiFiTTS/manifest_train_ngc.json
      sample_rate: 44100
      sup_data_path: /raid/sup_data_path_44100_new
      sup_data_types:
      - align_prior_matrix
      - pitch
      - speaker_id
      n_fft: 2048
      win_length: 2048
      hop_length: 512
      window: hann
      n

[NeMo I 2024-04-28 18:25:34 nemo_logging:381] PADDING: 1
[NeMo I 2024-04-28 18:25:35 nemo_logging:381] Model FastPitchModel was successfully restored from /home/you/.cache/torch/NeMo/NeMo_1.23.0/tts_en_fastpitch_multispeaker/b96451578e2b9a39d62cfc642bc5ffcf/tts_en_fastpitch_multispeaker.nemo.
[NeMo I 2024-04-28 18:25:35 nemo_logging:381] Found existing object /home/you/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo.
[NeMo I 2024-04-28 18:25:35 nemo_logging:381] Re-using file from: /home/you/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo
[NeMo I 2024-04-28 18:25:35 nemo_logging:381] Instantiating model from pre-trained checkpoint


[NeMo W 2024-04-28 18:25:37 nemo_logging:393] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.VocoderDataset
      manifest_filepath: /manifests/hifitts_fp_1000epoch_mels_train.json
      sample_rate: 44100
      n_segments: 16384
      max_duration: null
      min_duration: 0.75
      load_precomputed_mel: true
      hop_length: 512
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 100
      num_workers: 4
    
[NeMo W 2024-04-28 18:25:37 nemo_logging:393] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.Vocod

[NeMo I 2024-04-28 18:25:37 nemo_logging:381] PADDING: 0
[NeMo I 2024-04-28 18:25:37 nemo_logging:381] STFT using exact pad
[NeMo I 2024-04-28 18:25:37 nemo_logging:381] PADDING: 0
[NeMo I 2024-04-28 18:25:37 nemo_logging:381] STFT using exact pad


[NeMo W 2024-04-28 18:25:37 nemo_logging:393] /home/you/anaconda3/envs/tts_framework/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
      warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
    


[NeMo I 2024-04-28 18:25:38 nemo_logging:381] Model HifiGanModel was successfully restored from /home/you/.cache/torch/NeMo/NeMo_1.23.0/tts_en_hifitts_hifigan_ft_fastpitch/dcbdefa79d5e3587c93636d4f53260b8/tts_en_hifitts_hifigan_ft_fastpitch.nemo.


In [4]:
text_tts = """As the snake shook its head, a deafening shout behind Harry made both of them jump.
‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
"You’d be stiff if you’d been sitting on a brick wall all day," said Professor McGonagall.
"""

# All spectrogram generators start by parsing raw strings to a tokenized version of the string
parsed = spec_generator.parse(text_tts)

# Then take the tokenized string and produce a spectrogram
spectrogram = spec_generator.generate_spectrogram(tokens=parsed, speaker=9017)

# Finally, a vocoder converts the spectrogram to audio
audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)

# Save the audio to disk in a file called speech.wav
# sf.write("speech.wav", audio.detach().cpu().numpy(), 44100)

# Play the audio
Audio(audio.detach().cpu().numpy(), rate=44100)


[NeMo W 2024-04-28 18:27:30 nemo_logging:393] parse() is meant to be called in eval mode.
[NeMo W 2024-04-28 18:27:31 nemo_logging:393] Text: [as the SNEY1K SHUH1K its HHEH1D, a deafening SHAW1T BIH0HHAY1ND HHEH1RIY0 MEY1D BOW1TH of them JHAH1MP.‘DAH1DLIY0! MIH1STER0 dursley! KAH1M and LUH1K AE1T this SNEY1K! YUW1 WOW1NT BIH0LIY1V what it's DUW1IH0NG!'"you'd be STIH1F if you'd been SIH1TIH0NG on a BRIH1K WAO1L AO1L DEY1," SEH1D PRAH0FEH1SER0 mcgonagall.] contains unknown char/phoneme: [‘].Original text: [As the snake shook its head, a deafening shout behind Harry made both of them jump.‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON'T BELIEVE WHAT IT'S DOING!'"You'd be stiff if you'd been sitting on a brick wall all day," said Professor McGonagall.]. Symbol will be skipped.
[NeMo W 2024-04-28 18:27:31 nemo_logging:393] generate_spectrogram() is meant to be called in eval mode.


In [ ]:
text_tts = """As the snake shook its head, a deafening shout behind Harry made both of them jump.
"DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!"
"How did you know it was me?", she asked.
"My dear Professor, I’ve never seen a cat sit so stiffly".
"You’d be stiff if you’d been sitting on a brick wall all day", said Professor McGonagall.
"""

# text_tts = """As the snake shook its head, a deafening shout behind Harry made both of them jump."""

# text_tts = """‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’"""


# text_tts = """As the snake shook its head, a deafening shout behind Harry made both of them jump.
# ‘DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!’
# """

# text_tts1 = r"""As the snake shook its head, a deafening shout behind Harry made both of them jump.
# "DUDLEY! MR DURSLEY! COME AND LOOK AT THIS SNAKE! YOU WON’T BELIEVE WHAT IT’S DOING!"
# """

# text_tts2 = r""""How did you know it was me?" she asked.
# "My dear Professor, I’ve never seen a cat sit so stiffly."
# "You’d be stiff if you’d been sitting on a brick wall all day," said Professor McGonagall.
# """

def gen_audio(text_tts, speaker_id):
    # All spectrogram generators start by parsing raw strings to a tokenized version of the string
    parsed = spec_generator.parse(text_tts)
    # Then take the tokenized string and produce a spectrogram
    spectrogram = spec_generator.generate_spectrogram(tokens=parsed, speaker=speaker_id)
    
    # Finally, a vocoder converts the spectrogram to audio
    audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)

    return audio


html = f"""<table border='1'>
<h4>TTS: </h4> {text_tts}
<h4>Speakers: </h4>
<tr>
    <th>SpeakerID</th>
    <th>Audio</th>
    <th>VF Audio</th>
</tr>
"""


speakers = [
      92, #     Cori Samuel
    6097, #   Phil Benson
    9017, #   John Van Stan
    6670, #   Mike Pelton
    6671, #   Tony Oliva
    8051, #   Maria Kasper
    9136, #   Helen Taylor
    11614, #  Sylviamb
    11697, #  Celine Major
    12787, #  LikeManyWaters
]

for speaker_id in speakers:
    with torch.no_grad():
        audio = gen_audio(text_tts, speaker_id)
        # audio2 = gen_audio(text_tts2, speaker_id)

        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as input_file:
            sf.write(input_file.name, audio.squeeze(0).detach().cpu().numpy(), 44100)
            # Convert wav to mp3
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as output_file:
                voicefixer.restore(input=input_file.name, # low quality .wav/.flac file
                    output=output_file.name, # save file path
                    cuda=True, # GPU acceleration
                    mode=0,
                )
                # Read the wav file back into a numpy array
                wav_vf, samplerate = sf.read(output_file.name)

        html += f"""<tr>
            <td>{speaker_id}</td>
            <td>{Audio(audio.detach().cpu().numpy(), rate=44100)._repr_html_()}</td>
            <td>{Audio(wav_vf, rate=44100)._repr_html_()}</td>
        </tr>"""

# Save result as HTML
with open("logs/demo.html", "w") as f:
    f.write(html)

HTML(html)

In [5]:
from nemo.collections.tts.models import FastSpeech2HifiGanE2EModel

# Load the model from NGC
model = FastSpeech2HifiGanE2EModel.from_pretrained(model_name="tts_en_e2e_fastspeech2hifigan")

# Run inference
tokens = model.parse("Hey, I can speak!")
audio = model.convert_text_to_waveform(tokens=tokens)

ImportError: cannot import name 'FastSpeech2HifiGanE2EModel' from 'nemo.collections.tts.models' (/home/you/anaconda3/envs/tts_framework/lib/python3.11/site-packages/nemo/collections/tts/models/__init__.py)